In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import accessDB as db
from scipy import stats

pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.6.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 61 Stepping 4, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.23.4
pytest: 3.8.2
pip: 19.0.3
setuptools: 41.0.1
Cython: 0.28.5
numpy: 1.16.3
scipy: 1.1.0
pyarrow: None
xarray: None
IPython: 7.0.1
sphinx: 1.8.1
patsy: 0.5.0
dateutil: 2.7.3
pytz: 2018.5
blosc: None
bottleneck: 1.2.1
tables: 3.4.4
numexpr: 2.6.8
feather: None
matplotlib: 2.2.2
openpyxl: 2.5.8
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.1.1
lxml: 4.2.5
bs4: 4.6.3
html5lib: 0.9999999
sqlalchemy: 1.2.11
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: 0.7.0


In [6]:
# Read raw data in
df = db.filter(doGeoLoc=True)

In [7]:
df.head()

,Call Number,Date/Time,Location,Police District,Nature of Call,Status,Latitude,Longitude
ID,,,,,,,,
182551107-Unable to Locate Complainant,182551107,2018-09-12 11:51:11,"S 13TH ST / W MORGAN AV,MKE",6,ACC PDO,Unable to Locate Complainant,42.9811,-87.9289
182551111-Assignment Completed,182551111,2018-09-12 11:51:45,"500 E OAK ST, OAK CREEK",4,PROPERTY PICKUP,Assignment Completed,NaN,NaN
182551115-Assignment Completed,182551115,2018-09-12 11:53:49,"2333 N 49TH ST,MKE",3,RETURN STATION,Assignment Completed,43.0614,-87.9755
182551113-Advised/Referral,182551113,2018-09-12 11:54:07,"5348 N 91ST ST,MKE",4,ENTRY,Advised/Referral,43.1151,-88.0253
182551120-Service in Progress,182551120,2018-09-12 11:54:36,"6929 W SILVER SPRING DR,MKE",4,OUT OF SERVICE,Service in Progress,43.1189,-87.9982


In [8]:
district_status = df[['Status','Police District']].set_index('Police District')
district_status.head()

,Status
Police District,
6,Unable to Locate Complainant
4,Assignment Completed
3,Assignment Completed
4,Advised/Referral
4,Service in Progress


In [9]:
unique_statuses = district_status.Status.value_counts().index[:14].tolist()
unique_statuses

['Service in Progress',
 'Assignment Completed',
 'Advised',
 'Unable to Locate Complainant',
 'To be Filed',
 'City Citation(s) Issued',
 'Advised/Referral',
 'Open Investigation',
 'No Prosecution',
 'Cleared by Arrest',
 'Filed Driver Exchange Report',
 'False Alarm',
 'Referral',
 'Patrol Request']

In [10]:
district_status.Status.value_counts()

Service in Progress              157075
Assignment Completed             147855
Advised                           65488
Unable to Locate Complainant      31517
To be Filed                       16187
City Citation(s) Issued           12972
Advised/Referral                   8440
Open Investigation                 4192
No Prosecution                      506
Cleared by Arrest                   479
Filed Driver Exchange Report        333
False Alarm                         325
Referral                            116
Patrol Request                       80
False Alarm (Weather Related)        13
State Citation(s) Issued             10
Ordered to Appear                     1
Name: Status, dtype: int64

In [56]:
def dixon_qtest(data):
    data = sorted(data)
    q = 0.568
    
    gap = abs(data[1] - data[0])
    range = data[-1] - data[0]
    Q = gap/range
    if Q > q:
        # observation is an outlier
        print(str(data[1])+' is an outlier')
    print('min anomaly: '+str(round(Q,3))+' > '+str(q))
    gap = abs(data[-1] - data[-2])
    range = data[-1] - data[0]
    Q = gap/range
    if Q > q:
        # observation is an outlier
        print(str(data[1])+' is an outlier')
    print('max anomaly: '+str(round(Q,3))+' > '+str(q))

In [65]:
def std_from_mean(data):
    data = sorted(data)
    mean = sum(data)/len(data)
    
    total = 0
    for value in data:
        total += (value - mean)**2
    std = ((1/len(data)*total))**(1/2)
    
    for i, value in enumerate(data):
        difference_from_mean = abs(value - mean)
        std_from_mean = round(difference_from_mean/std, 3)
        print('value '+str(i+1)+' is '+str(std_from_mean)+' standard deviations from mean')
    

In [50]:
districts = ['1','2','3','4','5','6','7']
status_freq = pd.crosstab(df['Police District'],df['Status'])
status_freq = status_freq[status_freq.index.isin(districts)]
status_freq = status_freq[unique_statuses]
status_freq

Status,Service in Progress,Assignment Completed,Advised,Unable to Locate Complainant,To be Filed,City Citation(s) Issued,Advised/Referral,Open Investigation,No Prosecution,Cleared by Arrest,Filed Driver Exchange Report,False Alarm,Referral,Patrol Request
Police District,,,,,,,,,,,,,,
1,18766,16887,6465,1615,2100,1466,436,408,54,34,54,44,13,4
2,20088,21222,7888,4145,2341,1578,1211,494,54,86,62,36,22,13
3,29004,25413,9016,6168,2304,1463,1294,777,64,110,37,75,19,8
4,23158,21329,7665,4517,1974,1964,1303,609,82,56,54,53,10,15
5,22904,18442,12651,6209,2343,2209,1861,552,100,58,22,26,21,10
6,15066,19431,7943,2512,2088,1583,626,476,54,45,64,40,10,16
7,26851,24133,13608,6259,2930,2552,1703,866,98,87,39,51,20,14


In [51]:
total_statuses = status_freq.sum(axis=1).tolist()
status_percents = pd.DataFrame()

for index, total in enumerate(total_statuses):
    i = str(index+1)
    row = status_freq[status_freq.index == i]\
        .apply(lambda x: x/total_statuses[index],axis=0)
    status_percents = pd.concat([status_percents, row])

status_percents

Status,Service in Progress,Assignment Completed,Advised,Unable to Locate Complainant,To be Filed,City Citation(s) Issued,Advised/Referral,Open Investigation,No Prosecution,Cleared by Arrest,Filed Driver Exchange Report,False Alarm,Referral,Patrol Request
Police District,,,,,,,,,,,,,,
1,0.388160,0.349295,0.133724,0.033405,0.043437,0.030323,0.009018,0.008439,0.001117,0.000703,0.001117,0.000910,0.000269,0.000083
2,0.339095,0.358238,0.133153,0.069970,0.039517,0.026637,0.020442,0.008339,0.000912,0.001452,0.001047,0.000608,0.000371,0.000219
3,0.382881,0.335476,0.119020,0.081424,0.030415,0.019313,0.017082,0.010257,0.000845,0.001452,0.000488,0.000990,0.000251,0.000106
4,0.368823,0.339693,0.122076,0.071939,0.031439,0.031279,0.020752,0.009699,0.001306,0.000892,0.000860,0.000844,0.000159,0.000239
5,0.339782,0.273588,0.187678,0.092111,0.034758,0.032771,0.027608,0.008189,0.001484,0.000860,0.000326,0.000386,0.000312,0.000148
6,0.301597,0.388978,0.159006,0.050286,0.041798,0.031689,0.012532,0.009529,0.001081,0.000901,0.001281,0.000801,0.000200,0.000320
7,0.338981,0.304667,0.171794,0.079017,0.036990,0.032218,0.021500,0.010933,0.001237,0.001098,0.000492,0.000644,0.000252,0.000177


In [52]:
districts = ['1','2','3','4','5','6','7']
natures = list(df['Nature of Call'].value_counts()[:14].index)
nature_freq = pd.crosstab(df['Police District'],df['Nature of Call'])
nature_freq = nature_freq[nature_freq.index.isin(districts)]
nature_freq = nature_freq[natures]
nature_freq

Nature of Call,BUSINESS CHECK,TRAFFIC STOP,TRBL W/SUBJ,RETURN STATION,FOLLOW UP,PATROL,WELFARE CITIZEN,REPORTS,ACC PDO,INVESTIGATION,POLICE ADMIN,SPECIAL ASSIGN,SUSP PERS/AUTO,CALL FOR POLICE
Police District,,,,,,,,,,,,,,
1,6662,4424,3508,2208,1887,1478,1308,1328,1620,830,1886,1879,676,380
2,6098,4089,3494,3312,2319,2997,1567,1819,1962,1318,929,1097,1077,938
3,5411,4916,4194,5029,2975,1946,2016,2308,1943,3385,3636,2061,1227,1787
4,4677,4398,3272,3936,2160,4697,2106,1708,1570,1798,1551,1620,1357,1603
5,5201,8690,4088,2954,2170,1063,1943,2011,1434,2318,1039,1075,1013,1683
6,6849,4758,2711,2951,2068,3309,1751,1831,1690,629,1101,726,1562,515
7,9569,9911,4256,4488,2582,581,2274,1830,2053,1474,1296,836,1967,1671


In [53]:
total_natures = nature_freq.sum(axis=1).tolist()
nature_percents = pd.DataFrame()

for index, total in enumerate(total_natures):
    i = str(index+1)
    row = nature_freq[nature_freq.index == i]\
        .apply(lambda x: x/total_natures[index],axis=0)
    nature_percents = pd.concat([nature_percents, row])

nature_percents

Nature of Call,BUSINESS CHECK,TRAFFIC STOP,TRBL W/SUBJ,RETURN STATION,FOLLOW UP,PATROL,WELFARE CITIZEN,REPORTS,ACC PDO,INVESTIGATION,POLICE ADMIN,SPECIAL ASSIGN,SUSP PERS/AUTO,CALL FOR POLICE
Police District,,,,,,,,,,,,,,
1,0.221520,0.147104,0.116646,0.073419,0.062745,0.049145,0.043493,0.044158,0.053867,0.027599,0.062712,0.062479,0.022478,0.012635
2,0.184698,0.123849,0.105827,0.100315,0.070239,0.090774,0.047462,0.055094,0.059426,0.039920,0.028138,0.033226,0.032621,0.028410
3,0.126325,0.114769,0.097913,0.117407,0.069454,0.045431,0.047065,0.053882,0.045361,0.079026,0.084886,0.048116,0.028645,0.041719
4,0.128302,0.120649,0.089759,0.107975,0.059254,0.128851,0.057773,0.046855,0.043069,0.049324,0.042548,0.044441,0.037226,0.043974
5,0.141786,0.236901,0.111444,0.080530,0.059157,0.028979,0.052969,0.054823,0.039093,0.063192,0.028325,0.029306,0.027616,0.045881
6,0.211057,0.146621,0.083541,0.090937,0.063727,0.101969,0.053958,0.056424,0.052079,0.019383,0.033928,0.022372,0.048134,0.015870
7,0.213651,0.221287,0.095025,0.100205,0.057649,0.012972,0.050773,0.040859,0.045838,0.032911,0.028936,0.018666,0.043918,0.037309


In [57]:
for col in status_percents:
    print(col)
    dixon_qtest(status_percents[col].tolist())

Service in Progress
min anomaly: 0.432 > 0.568
max anomaly: 0.061 > 0.568
Assignment Completed
min anomaly: 0.269 > 0.568
max anomaly: 0.266 > 0.568
Advised
min anomaly: 0.045 > 0.568
max anomaly: 0.231 > 0.568
Unable to Locate Complainant
min anomaly: 0.288 > 0.568
max anomaly: 0.182 > 0.568
To be Filed
min anomaly: 0.079 > 0.568
max anomaly: 0.126 > 0.568
City Citation(s) Issued
min anomaly: 0.544 > 0.568
max anomaly: 0.041 > 0.568
Advised/Referral
min anomaly: 0.189 > 0.568
max anomaly: 0.329 > 0.568
Open Investigation
min anomaly: 0.055 > 0.568
max anomaly: 0.246 > 0.568
No Prosecution
min anomaly: 0.104 > 0.568
max anomaly: 0.278 > 0.568
Cleared by Arrest
min anomaly: 0.21 > 0.568
max anomaly: 0.001 > 0.568
Filed Driver Exchange Report
min anomaly: 0.17 > 0.568
max anomaly: 0.172 > 0.568
False Alarm
min anomaly: 0.367 > 0.568
max anomaly: 0.132 > 0.568
Referral
min anomaly: 0.193 > 0.568
max anomaly: 0.282 > 0.568
Patrol Request
min anomaly: 0.096 > 0.568
max anomaly: 0.343 > 0.56

In [58]:
for col in nature_percents:
    print(col)
    dixon_qtest(nature_percents[col].tolist())

BUSINESS CHECK
min anomaly: 0.021 > 0.568
max anomaly: 0.083 > 0.568
TRAFFIC STOP
min anomaly: 0.048 > 0.568
max anomaly: 0.128 > 0.568
TRBL W/SUBJ
min anomaly: 0.188 > 0.568
max anomaly: 0.157 > 0.568
RETURN STATION
min anomaly: 0.162 > 0.568
max anomaly: 0.214 > 0.568
FOLLOW UP
min anomaly: 0.12 > 0.568
max anomaly: 0.062 > 0.568
PATROL
min anomaly: 0.138 > 0.568
max anomaly: 0.232 > 0.568
WELFARE CITIZEN
min anomaly: 0.25 > 0.568
max anomaly: 0.267 > 0.568
REPORTS
min anomaly: 0.212 > 0.568
max anomaly: 0.085 > 0.568
ACC PDO
min anomaly: 0.196 > 0.568
max anomaly: 0.273 > 0.568
INVESTIGATION
min anomaly: 0.138 > 0.568
max anomaly: 0.265 > 0.568
POLICE ADMIN
min anomaly: 0.003 > 0.568
max anomaly: 0.391 > 0.568
SPECIAL ASSIGN
min anomaly: 0.085 > 0.568
max anomaly: 0.328 > 0.568
SUSP PERS/AUTO
min anomaly: 0.2 > 0.568
max anomaly: 0.164 > 0.568
CALL FOR POLICE
min anomaly: 0.097 > 0.568
max anomaly: 0.057 > 0.568


In [66]:
for col in status_percents:
    print(col)
    std_from_mean(status_percents[col].tolist())

Service in Progress
value 1 is 1.764 standard deviations from mean
value 2 is 0.438 standard deviations from mean
value 3 is 0.434 standard deviations from mean
value 4 is 0.41 standard deviations from mean
value 5 is 0.62 standard deviations from mean
value 6 is 1.119 standard deviations from mean
value 7 is 1.306 standard deviations from mean
Assignment Completed
value 1 is 1.797 standard deviations from mean
value 2 is 0.898 standard deviations from mean
value 3 is 0.007 standard deviations from mean
value 4 is 0.115 standard deviations from mean
value 5 is 0.393 standard deviations from mean
value 6 is 0.652 standard deviations from mean
value 7 is 1.541 standard deviations from mean
Advised
value 1 is 1.129 standard deviations from mean
value 2 is 1.004 standard deviations from mean
value 3 is 0.551 standard deviations from mean
value 4 is 0.528 standard deviations from mean
value 5 is 0.506 standard deviations from mean
value 6 is 1.029 standard deviations from mean
value 7 is 1.

In [67]:
for col in nature_percents:
    print(col)
    std_from_mean(nature_percents[col].tolist())

BUSINESS CHECK
value 1 is 1.253 standard deviations from mean
value 2 is 1.203 standard deviations from mean
value 3 is 0.858 standard deviations from mean
value 4 is 0.239 standard deviations from mean
value 5 is 0.913 standard deviations from mean
value 6 is 0.98 standard deviations from mean
value 7 is 1.181 standard deviations from mean
TRAFFIC STOP
value 1 is 0.953 standard deviations from mean
value 2 is 0.825 standard deviations from mean
value 3 is 0.756 standard deviations from mean
value 4 is 0.263 standard deviations from mean
value 5 is 0.252 standard deviations from mean
value 6 is 1.355 standard deviations from mean
value 7 is 1.694 standard deviations from mean
TRBL W/SUBJ
value 1 is 1.498 standard deviations from mean
value 2 is 0.933 standard deviations from mean
value 3 is 0.454 standard deviations from mean
value 4 is 0.192 standard deviations from mean
value 5 is 0.528 standard deviations from mean
value 6 is 1.038 standard deviations from mean
value 7 is 1.511 stan